- ver0.0 2025/10/27 雛形を作成 (小川)
- ver0.1 2025/11/01 章立てを作成 (小川)

<span style="color: red; ">**examples/2Q-calibrationのフォルダをご自身のworkspaceに移動させてから, 実験を行ってください.**</span>

## クラウド設定

一度だけ実行すれば良い.

In [ ]:
# from pathlib import Path

# config = """[default]
# url=https://api.qiqb-cloud.jp
# api_token=xxxxxxx
# """
# Path("~/.oqtopus").expanduser().write_text(config)
# print("Configuration saved.")

Configuration saved.


QIQBクラウドのリンク：https://www.qiqb-cloud.jp/  
  
「ジョブ」のタブで, 実行済み・実行中のジョブを確認することができます. 

## 2qubitの較正 

In [1]:
from quri_parts_oqtopus.backend import OqtopusSseBackend
from oqtopus_sse_pulse.adapter import QuriAdapter, collect_payloads_from_job, collect_session_from_job
from oqtopus_sse_pulse.plotters import plot_payload

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

DEVICE = "anemone" # デバイス名 (勉強会ではanemoneに固定)
backend = OqtopusSseBackend() # OqtopusSseBackendのインスタンスを作成

In [2]:
def experiment(program: str) -> list:
    """
    指定した実験プログラムを実行し, 測定結果のペイロードを返す関数.
    """

    # ジョブの実行
    job = backend.run_sse(
        file_path=f"{program}.py",
        device_id=f"{DEVICE}-pulse",
        name=program,
    )
    print(f"job_id = {job.job_id}")

    # ジョブの完了待ち
    job.wait_for_completion()
    print(f"job_status = {job.status}  elapsed={job.execution_time}")

    # QuriAdapterの作成
    adapter = QuriAdapter(backend)

    # 測定結果の取得
    payloads = collect_payloads_from_job(
        backend=adapter,
        job_id=job.job_id,
    )
    
    return payloads

### 使用する2qubitそれぞれの較正

事前に`1Q-calib.ipynb`を実行して, 使用する2つのqubitのhpiパルスの較正と読み出し分類器の調整を行っておいてください.

### 交差共鳴ゲート較正1：ZXハミルトニアンの生成 (TBD)

まずはcontrol = 0, 1の時にCRパルスだけ打って, tragetをX,Y,Z測定して観察. 

target qubitの回転は, $Z\times(c_xX+c_yY)とI(d_xX+d_yY)$によるもの.  
control qubitが0, 1の時, targetはそれぞれ$(\pm c_x+d_x)X+(\pm c_y+d_y)Y$の回転が生じる.  
ハミルトニアントモグラフィによって各係数を求めて, CRパルスの回転角の調整とキャンセルパルスの調整を行うことができる.   

IX, IY, IZ, ZX, ZY, ZZのうち, ZXのみが残ることを確かめる. 

### 交差共鳴ゲート較正2：ZX90パルスの較正 (TBD)

パルス長を調整してZX90回転に調整する

繰り返しZX90を打って, 確認.  
controlとtargetの役割の入れ替え. 

ZI90とIX90をつけると、CNOTが実現できる.  
繰り返し打って確認. 

### 2Q Randomized Benchmarking (TBD)

### 応用1：ベル状態生成 (TBD)

### 応用2：ベルの不等式の破れの確認 (TBD)

### 応用3：2qubit VQEの実装 (TBD)